# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
# selenium permite controlar o navegador de forma automática
# instalar o chrome driver https://chromedriver.chromium.org/downloads
#!pip install selenium

from selenium import webdriver # controlar o navegador
from selenium.webdriver.common.keys import Keys # controlar o teclado
from selenium.webdriver.common.by import By # Localizar os items no navegador

#cria o navegador
navegador = webdriver.Chrome()

# entraria no google e pesquisaria por cotação do dólar
navegador.get("https://www.google.com/")

# pegaria a cotação do dólar - clica no elemento do campo de busca e escreve "cotação dólar"
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print(cotacao_dolar)

# entraria no google e pesquisaria a cotação do euro
navegador.get("https://www.google.com/")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print(cotacao_euro)

# entraria no site https://www.melhorcambio.com/ouro-hoje
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute("value") 
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

5.0213
5.521873397
324.63


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
# importar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

Produtos  Preço Original  Moeda  Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar        5          4999.95    1.40   
1   Carro Renault         4500.00   Euro        6         27000.00    2.00   
2   Notebook Dell          899.99  Dólar        5          4499.95    1.70   
3          IPhone          799.00  Dólar        5          3995.00    1.70   
4      Carro Fiat         3000.00   Euro        6         18000.00    1.90   
5  Celular Xiaomi          480.48  Dólar        5          2402.40    2.00   
6        Joia 20g           20.00   Ouro      350          7000.00    1.15   

   Preço de Venda  
0        6999.930  
1       54000.000  
2        7649.915  
3        6791.500  
4       34200.000  
5        4804.800  
6        8050.000

- Atualizando os preços e o cálculo do Preço Final

In [3]:
# atualizar as cotações na base de dados
# atualizar o preço de compra e de venda na base de dados

# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)


Produtos  Preço Original  Moeda     Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar    5.021300      5021.249787    1.40   
1   Carro Renault         4500.00   Euro    5.521873     24848.430287    2.00   
2   Notebook Dell          899.99  Dólar    5.021300      4519.119787    1.70   
3          IPhone          799.00  Dólar    5.021300      4012.018700    1.70   
4      Carro Fiat         3000.00   Euro    5.521873     16565.620191    1.90   
5  Celular Xiaomi          480.48  Dólar    5.021300      2412.634224    2.00   
6        Joia 20g           20.00   Ouro  324.630000      6492.600000    1.15   

   Preço de Venda  
0     7029.749702  
1    49696.860573  
2     7682.503638  
3     6820.431790  
4    31474.678363  
5     4825.268448  
6     7466.490000

### Agora vamos exportar a nova base de preços atualizada

In [4]:
# exportar essa base de dados para ter o resultado atualizado
tabela.to_excel("Produtos Novo.xlsx", index=False)